In [1]:
# imports
from sqlalchemy import create_engine, Column, String, Integer, Numeric, MetaData, Table, type_coerce
from sqlalchemy.orm import mapper, create_session
import csv
from sqlalchemy import case
import numpy as np
from sqlalchemy.ext.automap import automap_base
import matplotlib.pyplot as plt
import sqlalchemy.types as types
import numbers
from sqlalchemy.sql import and_, or_, not_
from sqlalchemy.sql import select, func

import matplotlib.lines as mlines
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.basemap import Basemap


%matplotlib inline

In [2]:
engine = create_engine('sqlite:///DFOOceanProfileDB/salishDB.sqlite')

In [3]:
Base = automap_base()
# reflect the tables in salish.sqlite:
Base.prepare(engine, reflect=True)
# mapped classes have been created
Profs=Base.classes.profiles

In [4]:
session = create_session(bind = engine, autocommit = False, autoflush = True)

In [5]:
#sal=case([(Profs.SSAL!=None, Profs.SSAL)], else_=Profs.PSAL)
#data=session.query(sal, Profs.NTRA).all()
#.filter(Profs.DEPTH_PRESS>100).filter(Profs.DEPTH_PRESS!=None),.filter(sal!=None).filter(Profs.NTRA!=None)
data=session.query(Profs.Q_DATE_TIME).filter(Profs.CR_NUMBER=='18DD12014 ').filter(Profs.OBS_TIME==724).all()
print(data)


[(None,), (None,), (None,), (None,), (None,), (None,)]


/home/eolson/anaconda3/lib/python3.4/site-packages/sqlalchemy/sql/sqltypes.py:562: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  'storage.' % (dialect.name, dialect.driver))


In [6]:
datanum=session.query(Profs).count()
print(datanum)

35785


In [ ]:
sal=case([(Profs.SSAL!=None, Profs.SSAL)], else_=Profs.PSAL)
qsal=case([(Profs.SSAL!=None, Profs.Q_SSAL)], else_=Profs.Q_PSAL)
data=session.query(sal, Profs.NTRA)
print('Q_SAL')
print(session.query(qsal).group_by(qsal).all())
print('Q_NO3')
print(session.query(Profs.Q_NTRA).group_by(Profs.Q_NTRA).all())
NO=[]
S=[]
for m, n in data.all():
    S.append(m)
    NO.append(n)

In [ ]:
data=session.query(Profs.LONGITUDE, Profs.LATITUDE).filter(Profs.LONGITUDE!=None,Profs.LATITUDE!=None)
Lon=[]
Lat=[]
for m, n in data.all():
    Lon.append(float(-m))
    Lat.append(float(n))
    

plt.plot(Lon,Lat,'.')

In [ ]:
### Draw the map for the location of the stations

fig,ax=plt.subplots(1,1,facecolor='w',figsize=(10,10))
x=100000
ma = Basemap(llcrnrlon=-130, llcrnrlat=46, urcrnrlon=-120, urcrnrlat=52,lon_0=-100, resolution='l',area_thresh=1,projection='merc')
ma.drawcoastlines()
ma.drawcountries()
ma.fillcontinents(color='coral')

#m.drawmeridians(np.arange(-160,-50,20), labels=[0,0,0,1])
#m.drawparallels(np.arange(65,85,5), labels=[1,0,0,0])

cmap = plt.get_cmap('jet')
data=session.query(Profs.LONGITUDE, Profs.LATITUDE).filter(Profs.LONGITUDE!=None,Profs.LATITUDE!=None)
Lon=[]
Lat=[]
for m, n in data.all():
    Lon.append(float(m))
    Lat.append(float(n))
Lon2=np.array(Lon)[:]
Lat2=np.array(Lat)[:]
x, y = ma(-Lon2, Lat2)
ma.scatter(x,y)


In [ ]:
plt.plot(S,NO,'.')
plt.show()

In [ ]:
data2=session.query(sal, Profs.NTRA).filter(Profs.DEPTH_PRESS>400)
NO=[]
S=[]
for m, n in data2.all():
    S.append(m)
    NO.append(n)
    
plt.axis([0,35,0,100])
plt.plot(S,NO,'.')

In [ ]:
data2=session.query(sal, Profs.NTRA).filter(and_(Profs.DEPTH_PRESS<400,Profs.DEPTH_PRESS>100))
NO=[]
S=[]
for m, n in data2.all():
    S.append(m)
    NO.append(n)
    
plt.axis([0,35,0,100])
plt.plot(S,NO,'.')

In [ ]:
data2=session.query(sal, Profs.NTRA).filter(and_(Profs.DEPTH_PRESS<100,Profs.DEPTH_PRESS>50))
NO=[]
S=[]
for m, n in data2.all():
    S.append(m)
    NO.append(n)
    
plt.axis([0,35,0,100])
plt.plot(S,NO,'.')

In [ ]:
data2=session.query(sal, Profs.NTRA).filter(and_(Profs.DEPTH_PRESS<50,Profs.DEPTH_PRESS>25))
NO=[]
S=[]
for m, n in data2.all():
    S.append(m)
    NO.append(n)
    
plt.axis([0,35,0,100])
plt.plot(S,NO,'.')

In [ ]:
data2=session.query(sal, Profs.NTRA).filter(and_(Profs.DEPTH_PRESS<25,Profs.DEPTH_PRESS>=0))
NO=[]
S=[]
for m, n in data2.all():
    S.append(m)
    NO.append(n)
    
plt.axis([0,35,0,100])
plt.plot(S,NO,'.')

In [ ]:
q1=session.query(Profs.OBS_YEAR, func.count(Profs.OBS_YEAR)).filter(
    and_(
        Profs.LATITUDE > 48.2,
        Profs.LATITUDE < 48.8,
        Profs.LONGITUDE < 125.2,
        Profs.LONGITUDE > 124.2,
        Profs.DEPTH_PRESS > 50.0,
        sal != 'NaN',
        Profs.NTRA != 'NaN',
        Profs.TEMP != 'NaN'
        )).group_by(Profs.OBS_YEAR).all()
print(q1)

In [ ]:
data2=session.query(sal, Profs.NTRA, Profs.TEMP, Profs.LONGITUDE, Profs.OBS_YEAR, Profs.OBS_MONTH, Profs.OBS_DAY, Profs.DEPTH_PRESS).filter(
    and_(
        Profs.LATITUDE > 48.2,
        Profs.LATITUDE < 48.8,
        Profs.LONGITUDE < 125.2,
        Profs.LONGITUDE > 124.2,
        Profs.DEPTH_PRESS > 50.0,
        sal != 'NaN',
        Profs.NTRA != 'NaN',
        Profs.TEMP != 'NaN'
        ))
NO=[]
S=[]
T=[]
Lon=[]
Yr=[]
Mon=[]
Day=[]
Z=[]
for m, n, o, p, q, r, s, t in data2.all():
    S.append(float(m))
    NO.append(float(n))
    T.append(float(o))
    Lon.append(float(p))
    Yr.append(float(q))
    Mon.append(float(r))
    Day.append(float(s))
    Z.append(float(t))
    
plt.axis([28,35,0,40])
plt.plot(S,NO,'.')
plt.xlabel('Salinity')
plt.ylabel('NO')

In [ ]:
NO=np.array(NO)
S=np.array(S)
T=np.array(T)

Lon=np.array(Lon)
Yr=np.array(Yr)
Mon=np.array(Mon)
Day=np.array(Day)
Z=np.array(Z)

In [ ]:
a=np.vstack([T,S,np.ones(len(T))]).T
m = np.linalg.lstsq(a,NO)[0]
mT, mS, m0 = m
print(mT, mS, m0)

In [ ]:
plt.plot(NO,np.dot(a,m),'.')
plt.xlabel('NO')
plt.ylabel('NO, linear model')
SSE=np.dot((NO-np.dot(a,m)),(NO-np.dot(a,m)).T)
plt.title('Linear Model: NO(T,S)')
print('SSE=', SSE)
plt.plot(np.array([0,40]), np.array([0, 40]),'r-')

In [ ]:
plt.plot(Z,NO-np.dot(a,m),'.')
plt.xlabel('Depth')
plt.ylabel('Residuals')
plt.title('Linear Model: NO(T,S)')

In [ ]:
plt.plot(Lon,NO-np.dot(a,m),'.')
plt.xlabel('Lon')
plt.ylabel('Residuals')
plt.title('Linear Model: NO(T,S)')

In [ ]:
plt.plot(Mon+Day/30.0,NO-np.dot(a,m),'.') # Day/30 = cheat to get approx fraction of month
plt.xlabel('Month')
plt.ylabel('Residuals')
plt.title('Linear Model: NO(T,S)')

In [ ]:
plt.plot(Yr+Mon/12.0+Day/365.0,NO-np.dot(a,m),'.') # Day/30 = cheat to get approx fraction of month
plt.xlabel('Yearr')
plt.ylabel('Residuals')
plt.title('Linear Model: NO(T,S)')

In [ ]:
a=np.vstack([T,np.ones(len(T))]).T
m = np.linalg.lstsq(a,NO)[0]
print('m=', m)
plt.plot(NO,np.dot(a,m),'.')
plt.xlabel('NO')
plt.ylabel('NO, linear model')
SSE=np.dot((NO-np.dot(a,m)),(NO-np.dot(a,m)).T)
plt.title('Linear Model: NO(T)')
print('SSE=', SSE)
plt.plot(np.array([0,40]), np.array([0, 40]),'r-')

In [ ]:
a=np.vstack([S,np.ones(len(T))]).T
m = np.linalg.lstsq(a,NO)[0]
print('m=', m)
plt.plot(NO,np.dot(a,m),'.')
plt.xlabel('NO')
plt.ylabel('NO, linear model')
SSE=np.dot((NO-np.dot(a,m)),(NO-np.dot(a,m)).T)
plt.title('Linear Model: NO(S)')
print('SSE=', SSE)
plt.plot(np.array([0,40]), np.array([0, 40]),'r-')

In [ ]:

import matplotlib.lines as mlines
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.basemap import Basemap

### Draw the map for the location of the stations

#m = Basemap(llcrnrlon=-170, llcrnrlat=65, urcrnrlon=-50, urcrnrlat=85,lon_0=-100, resolution='l',area_thresh=10,projection='merc')
fig,ax=plt.subplots(1,1,facecolor='w',figsize=(10,10))
x=100000
m = Basemap(llcrnrx=-20*x, llcrnry=-10*x, urcrnrx=+15*x, urcrnry=+15*x,lon_0=-100, lat_0=75,projection='ortho',resolution='l')
m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color='coral')

m.drawmeridians(np.arange(-160,-50,20), labels=[0,0,0,1])
m.drawparallels(np.arange(65,85,5), labels=[1,0,0,0])

cmap = plt.get_cmap('jet')


In [ ]:
session.close()
engine.dispose()